In [65]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# import requests

url_toms_hardware = 'https://www.tomshardware.com/reviews/gpu-hierarchy,4388.html'
# req_toms_hardware = requests.get(url_toms_hardware)
# req_toms_hardware

In [3]:
# print(req_toms_hardware.text[:500])

In [4]:
# import bs4
# soup = bs4.BeautifulSoup(req_toms_hardware.text, 'html.parser')

In [5]:
# gpu_rasterization_table = soup.find('div', attrs={'id':'slice-container-table-L8SfcAV5o28ayEFjzq6cfX-10'}).find('div', class_='table__container')

In [6]:
# gpu_rasterization_headers = []
# for th in gpu_rasterization_table.find('thead').find_all('th'):
#     gpu_rasterization_headers.append(th.text)
# gpu_rasterization_headers

In [7]:
# gpu_rasterization_body = []
# for tr in gpu_rasterization_table.find('tbody').find_all('tr'):
#     cells = []
#     for td in tr.find_all('td'):
#         cells.append(td.text if td.text != '' else np.NaN)
#     gpu_rasterization_body.append(cells)
# gpu_rasterization_body[:3]

In [8]:
# gpu_df = pd.DataFrame(gpu_rasterization_body, columns=gpu_rasterization_headers).drop('Specifications (Links to Review)', axis=1)
# gpu_df

In [26]:
# could use the pd.read_html function instead of bs4
tables = pd.read_html(url_toms_hardware)

In [55]:
gpu_rasterization_df = tables[0]
gpu_rasterization_df.head()

,Graphics Card,Lowest Price,1080p Ultra,1080p Medium,1440p Ultra,4K Ultra,Specifications (Links to Review)
0,GeForce RTX 4090,$1849,100.0% (154.1fps),100.0% (195.7fps),100.0% (146.1fps),100.0% (114.5fps),"AD102, 16384 shaders, 2520MHz, 24GB GDDR6X@21G..."
1,Radeon RX 7900 XTX,$909,96.7% (149.0fps),97.2% (190.3fps),92.6% (135.3fps),83.1% (95.1fps),"Navi 31, 6144 shaders, 2500MHz, 24GB GDDR6@20G..."
2,GeForce RTX 4080 Super,$999,96.2% (148.3fps),98.5% (192.7fps),91.0% (133.0fps),80.3% (91.9fps),"AD103, 10240 shaders, 2550MHz, 16GB GDDR6X@23G..."
3,GeForce RTX 4080,$1185,95.4% (147.0fps),98.1% (192.0fps),89.3% (130.4fps),78.0% (89.3fps),"AD103, 9728 shaders, 2505MHz, 16GB GDDR6X@22.4..."
4,Radeon RX 7900 XT,$699,93.4% (143.9fps),95.8% (187.6fps),86.1% (125.9fps),71.0% (81.2fps),"Navi 31, 5376 shaders, 2400MHz, 20GB GDDR6@20G..."


In [56]:
gpu_rasterization_df.describe()

,Graphics Card,Lowest Price,1080p Ultra,1080p Medium,1440p Ultra,4K Ultra,Specifications (Links to Review)
count,86,44,82,86,71,42,86
unique,86,38,81,84,70,42,86
top,GeForce RTX 4090,$399,84.7% (130.5fps),26.1% (51.1fps),22.8% (33.3fps),100.0% (114.5fps),"AD102, 16384 shaders, 2520MHz, 24GB GDDR6X@21G..."
freq,1,3,2,2,2,1,1


- Some missing values in Lowest Price, 1080p Ultra, 1440p Ultra, and 4K Ultra. 
- Specifications don't mean anything to us
- Remove the '$' from Lowest Price
- For the resolution categories, we need to separate percentiles from raw fps

In [59]:
def price_to_float(df, column_name):
    df[column_name] = df[column_name].str.replace('$', '').astype(float)
    return df

def get_fps(df, column_names: list[str]):
    for column_name in column_names:
        df[column_name] = gpu_rasterization_df[column_name].str.split(' ', expand=True)[1].str.replace(r'[^\d.]+', '', regex=True).astype(float)
    return df

gpu_rasterization_df = (gpu_rasterization_df
                        .drop('Specifications (Links to Review)', axis=1)
                        .pipe(price_to_float, column_name='Lowest Price')
                        .pipe(get_fps, column_names=['1080p Ultra', '1080p Medium', '1440p Ultra', '4K Ultra']))
gpu_rasterization_df.head()

,Graphics Card,Lowest Price,1080p Ultra,1080p Medium,1440p Ultra,4K Ultra
0,GeForce RTX 4090,1849.0,154.1,195.7,146.1,114.5
1,Radeon RX 7900 XTX,909.0,149.0,190.3,135.3,95.1
2,GeForce RTX 4080 Super,999.0,148.3,192.7,133.0,91.9
3,GeForce RTX 4080,1185.0,147.0,192.0,130.4,89.3
4,Radeon RX 7900 XT,699.0,143.9,187.6,125.9,81.2


In [86]:
px.scatter(gpu_rasterization_df,
           x= 'Lowest Price',
           y= '1080p Ultra',
           hover_name='Graphics Card',
           title='GPU Rasterization For 1080p Ultra')

In [88]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=2, 
                    shared_yaxes=True,
                    subplot_titles=("1080p Ultra", "1080p Medium", "1440p Ultra", "4K Ultra"))

fig.add_trace(go.Scatter(x=gpu_rasterization_df['Lowest Price'], y=gpu_rasterization_df['1080p Ultra'], mode='markers', hovertext=gpu_rasterization_df['Graphics Card'], name='1080 Ultra'), row=1, col=1)
fig.add_trace(go.Scatter(x=gpu_rasterization_df['Lowest Price'], y=gpu_rasterization_df['1080p Medium'], mode='markers', hovertext=gpu_rasterization_df['Graphics Card'], name='1080p Medium'), row=1, col=2)
fig.add_trace(go.Scatter(x=gpu_rasterization_df['Lowest Price'], y=gpu_rasterization_df['1440p Ultra'], mode='markers', hovertext=gpu_rasterization_df['Graphics Card'], name='1440p Ultra'), row=2, col=1)
fig.add_trace(go.Scatter(x=gpu_rasterization_df['Lowest Price'], y=gpu_rasterization_df['4K Ultra'], mode='markers', hovertext=gpu_rasterization_df['Graphics Card'], name='4K Ultra'), row=2, col=2)

fig.update_layout(title='Scatter plots of price vs resolutions',
                  xaxis_title='Price',
                  yaxis_title='FPS')
fig.show()